# Section of region of interest

Load the file at a low resolution and add regions of interest.

In [ ]:
import yaml
from pathlib import Path
from ipyfilechooser import FileChooser
srcdir, dstdir = '', ''
if Path("config.yml").exists():
    with open("config.yml", "r") as file:    
        config = yaml.safe_load(file)
        if 'source' in config.keys():
            srcdir = Path(config["source"])        
        if 'destination' in config.keys():
            dstdir = Path(config["destination"]) 

fc = FileChooser(dstdir, select_desc='destination')
display(fc)



Check the list of files defined at step 1

In [ ]:
import pandas as pd
dstdir = Path(fc.selected) if fc.selected is not None else Path(dstdir)
filelistname = dstdir / 'filelist.csv'
filelist = pd.read_csv(filelistname)
filelist

Select a file from the list

In [ ]:
import ipywidgets as widgets
w = widgets.Dropdown(
    options=[(x,k) for k,x in enumerate(filelist['name'])],
    value=1,
    description='Image:',
)
display(w)

Load the selected file and update config.yml

In [ ]:
def get_files(dstdir, row, key=None):
    if key == 'ims':
        return Path(row['folder']) / row['name']
    elif key == 'regions':
        return Path(dstdir / str(row["name"]).replace('.ims','-regions.json'))
    elif key == 'labels':
        return Path(dstdir / str(row['name']).replace('.ims','-labels.tif'))
    elif key == 'measurements':
        return Path(dstdir / str(row['name']).replace('.ims','-measurements.csv'))    
    else:
        return {
            'ims': get_files(dstdir, row, 'ims'),
            'regions': get_files(dstdir, row, 'regions'),
            'labels': get_files(dstdir, row, 'labels'),
            'measurements':  get_files(dstdir, row, 'measurements')
        }
    
row = filelist.iloc[w.value]
filename = get_files(dstdir, row, 'ims')
regionname = get_files(dstdir, row, 'regions')
print("Input IMS filename:", filename)
print("Destination:", dstdir)
print("Output region file:", regionname)
# update yaml config file
try:
    cnf = yaml.safe_load(f'source: \"{srcdir.as_posix()}\"\ndestination: \"{dstdir.as_posix()}\"\n')
    with open('config.yml', 'w') as file:
        yaml.dump(cnf, file)
except:
    pass


Load the file from disk, change the resolution level if needed

In [ ]:
from imaris_ims_file_reader.ims import ims
import dask.array as da

resolution_level = 1

store = ims(filename, ResolutionLevelLock=resolution_level, aszarr=True)

img = da.from_zarr(store)
nuclei = da.max(img[0,3],0).compute().squeeze()
print(nuclei.shape)

Load the file in napari and add some regions. If there are existing regions, load the regions as shapes.

In [ ]:
import json
import numpy as np
import napari

v = napari.view_image(nuclei)
if regionname.exists():
    with open(regionname, "r") as outfile:
        poly = json.load(outfile)
    c = pow(2, poly['resolution_level']- resolution_level)    
    sdata = [c * np.array(p) for p in poly['regions']]
    v.add_shapes(sdata,shape_type='polygon')


Export the region to a json file

In [ ]:
import json
poly = {
    'shape': nuclei.shape,
    'resolution_level': resolution_level,
    'regions': [ p.tolist() for p in v.layers[-1].data ] 
}

with open(regionname, "w") as outfile:
    json.dump(poly, outfile)

Reload the regions to check for correctness

In [ ]:
import matplotlib.pyplot as plt

with open(regionname, "r") as outfile:
    poly2 = json.load(outfile)

rois = napari.layers.Shapes(poly2['regions'],shape_type='polygon').to_labels(labels_shape=poly2['shape'])
plt.imshow(nuclei)
plt.imshow(rois,alpha=0.5)
plt.axis('off');